<a href="https://colab.research.google.com/github/rajagopal17/Text-Analysis-with-Spacy/blob/master/DistilBert_imDB_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##http://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/

In [ ]:
!pip3 install ktrain

In [4]:
import tensorflow as tf

In [3]:
tf.__version__

'2.1.0'

In [4]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [5]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [6]:
data_train =pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx',dtype=str)

In [7]:
data_test =pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype=str)

In [8]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


##STEP 1: Load and Preprocess the Dataset
The texts_from_folder function will load the training and validation data from the specified folder and automatically preprocess it according to BERT's requirements. In doing so, the BERT model and vocabulary will be automatically downloaded.

In the below step we get 3 parameters as output. That is converted train & test data as per BERT and also data is preprocessed





In [9]:
train,val,preproc= text.texts_from_df(train_df=data_train,text_column='Reviews',label_columns='Sentiment',val_df=data_test,maxlen=512,preprocess_mode='distilbert')

preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


##STEP 2: Load a pretrained BERT model 

In [10]:
model=text.text_classifier(name='distilbert',train_data=(train),preproc=preproc)

Is Multi-Label? False
maxlen is 512



done.


##STEP 3: Wrap the bert model with ktrain.learner object, that is instantiate a learner 

In [11]:
learner=ktrain.get_learner(model=model,train_data=(train),val_data=(val),batch_size=6)

##STEP 4: Train and Fine-Tune the Model on the IMDb dataset
We employ the learner.fit_onecycle method in ktrain that employs the use of a 1cycle learning rate schedule. We use a learning rate of 2e-5 based on recommendations from the original paper.

We train using one of the three learning rates recommended in the BERT paper: 5e-5, 3e-5, or 2e-5. Alternatively, the ktrain Learning Rate Finder can be used to find a good learning rate by invoking learner.lr_find() and learner.lr_plot(), prior to training. The learner.fit_onecycle method employs a 1cycle learning rate policy.

In [12]:
learner.fit_onecycle(lr=2e-5,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
Train for 4167 steps, validate for 782 steps
4167/4167 [==============================] - 3967s 952ms/step - loss: 0.2701 - accuracy: 0.8862 - val_loss: 0.1258 - val_accuracy: 0.9592


##We can use the learner.validate method to test our model against the validation set.

In [ ]:
learner.validate(val_data=val,print_report=True,class_names=['neg','pos'])

##How to Use Our Trained BERT Model
We can call the learner.get_predictor method to obtain a Predictor object capable of making predictions on new raw data.

In [16]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [17]:
predictor.get_classes()

['neg', 'pos']

In [18]:
predictor.save('/content/drive/My Drive/Python/DistilBert')

##Enter text data for prediction

In [28]:
data=['was never a fan before but now iam ','not worth the dime though ']

In [29]:
predictor.predict(data)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:229: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  'If this is a sentence pair classification task, please cast to tuple.')


['pos', 'neg']

##Load a saved model

In [6]:

bertmodel=ktrain.load_predictor('/content/drive/My Drive/Python/DistilBert/', batch_size=32)

##Input text for getting predictions

In [7]:
ex=['value for money','did not connect with audience']

##Get prediction for entered text

In [10]:
bertmodel.predict(ex)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:229: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  'If this is a sentence pair classification task, please cast to tuple.')


['pos', 'neg']